### Disclaimer

Distribution authorized to U.S. Government agencies and their contractors. Other requests for this document shall be referred to the MIT Lincoln Laboratory Technology Office.

This material is based upon work supported by the Under Secretary of Defense for Research and Engineering under Air Force Contract No. FA8702-15-D-0001. Any opinions, findings, conclusions or recommendations expressed in this material are those of the author(s) and do not necessarily reflect the views of the Under Secretary of Defense for Research and Engineering.

© 2019 Massachusetts Institute of Technology.

The software/firmware is provided to you on an As-Is basis

Delivered to the U.S. Government with Unlimited Rights, as defined in DFARS Part 252.227-7013 or 7014 (Feb 2014). Notwithstanding any copyright notice, U.S. Government rights in this work are defined by DFARS 252.227-7013 or DFARS 252.227-7014 as detailed above. Use of this work other than as specifically authorized by the U.S. Government may violate any copyrights that exist in this work.


In [ ]:
from tesse.env import Env
from tesse.msgs import *

import time
import matplotlib.pyplot as plt
import defusedxml.ElementTree as ET
import numpy as np

In [ ]:
env = Env()

### Demo Transform Message

In [ ]:
env.send(Transform(-1, 0, 0)) # left
time.sleep(1)
env.send(Transform(1, 0, 0)) # right
time.sleep(1)
env.send(Transform(0, -1, 0)) # back
time.sleep(1)
env.send(Transform(0, 1, 0)) # forward
time.sleep(1)
env.send(Transform(0, 0, 45)) # turn right
time.sleep(1)
env.send(Transform(0, 0, -45)) # turn left

### Demo Force and Torque Message

In [ ]:
env.send(AddForce(10, 0)) # Forward force
time.sleep(1)
env.send(AddForce(-10, 0)) # Backward force
time.sleep(1)
env.send(AddForce(0, 1)) # Torque right
time.sleep(1)
env.send(AddForce(0, -1)) # Torque left

### Demo Reposition Message

In [ ]:
env.send(Reposition(0, .5, 2, 0, 0, 0, 0))
time.sleep(1)
env.send(Reposition(2, 2, 0, 0, 0, 0, 0))

### Demo getting images

In [ ]:
def plot(response, title=None):
    if len(response.images) == 0:
        return
    figsize = (len(response.images)*5, 5)
    fig, axs = plt.subplots(1, len(response.images), figsize=figsize)
    if len(response.images) > 1:
        for ax, img in zip(axs, response.images):
            ax.axis('off');
            ax.imshow(img, cmap='gray');
    else:
        axs.axis('off');
        axs.imshow(response.images[0], cmap='gray');
        axs.set_title(title)

In [ ]:
response = env.request(DataRequest())
if response is not None:
    print(response.metadata)
    plot(response)

In [ ]:
cameras=[
    (Camera.RGB_LEFT, Compression.OFF, Channels.THREE),
    (Camera.SEGMENTATION, Compression.OFF, Channels.THREE),
]
response = env.request(DataRequest(metadata=False, cameras=cameras))
if response is not None:
    plot(response)

### Metadata request

In [ ]:
response = env.request(MetadataRequest())
if response is not None:
    print(response.metadata)

### Camera request

In [ ]:
response = env.request(CameraInformationRequest())
if response is not None:
    print(response.metadata)

### Demonstrate some camera manipulation

In [ ]:
# Reset 3rd person view
_ = env.request(SetCameraParametersRequest(Camera.THIRD_PERSON, 768, 1024, 90, .05, 10))
_ = env.request(SetCameraPositionRequest(Camera.THIRD_PERSON, 0, 1.319, -2.258))
response = env.request(DataRequest(cameras=[(Camera.THIRD_PERSON, Compression.OFF, Channels.THREE)]))
if response is not None:
    plot(response, 'First Parameters')

# Change the parameters and get the camera
_ = env.request(SetCameraParametersRequest(Camera.THIRD_PERSON, 768, 1024, 55, .3, 50))
response = env.request(DataRequest(cameras=[(Camera.THIRD_PERSON, Compression.OFF, Channels.THREE)]))
if response is not None:
    plot(response, 'Second Parameters')


# Change the position and get the camera
_ = env.request(SetCameraPositionRequest(Camera.THIRD_PERSON, 0, 0.5, -2.5))
response = env.request(DataRequest(cameras=[(Camera.THIRD_PERSON, Compression.OFF, Channels.THREE)]))
if response is not None:
    plot(response, 'New Position')

### Demo respawning

In [ ]:
for i in range(3):
    env.send(Respawn())
    response = env.request(DataRequest(cameras=[(Camera.THIRD_PERSON, Compression.OFF, Channels.THREE)]))
    if response is not None:
        plot(response)
    time.sleep(1)

### Demo changing scenes

*NOTE: loading scenes for the first time may take a long time*

In [ ]:
scene_index = 7

response = env.request(SceneRequest(scene_index))
if response is not None:
    print(response.metadata)

### Demo step mode

This puts the game into step mode. The rate defines the update rate (Hz) for the game.

Then, we step forward several times showing how Unit time progresses.

In [ ]:
rate = 1
env.send(SetFrameRate(rate))

In [ ]:
def unity_time(env):
    response = env.request(MetadataRequest())
    root = ET.fromstring(response.metadata)
    return float(root.find('time').text)

In [ ]:
unity_start = unity_time(env)

for _ in range(10):
    env.send(StepWithForce())
    print("Elapsed time is ", unity_time(env) - unity_start, " seconds.")

In [ ]:
# Exit step mode by setting rate to 0
rate = 0
env.send(SetFrameRate(rate))

### Demo colliders

Turn collisions off, then back on

In [ ]:
env.send(ColliderRequest(enable=0)) # turn collisions off

In [ ]:
env.send(ColliderRequest(enable=1)) # turn collisions on

### Demo Manipulation of Objects
This moves the agent then spawn 36 cubes around the agent

In [ ]:
x = -6
y = .5
z = 0 #8.5
radius = 1.5
orientation = [ 0.4619398, 0.1913417, 0.4619398, 0.7325378 ]
orientation = [0, 0, 0, 1]
env.send(Reposition(x, y, z, 0, 0, 0, 0))
time.sleep(.2)

for angle in range(0, 360, 10):
    response = env.request(SpawnObjectRequest(int((angle % 40)/10),
                                ObjectSpawnMethod.USER,
                                x + radius*np.cos(angle*np.pi/180), 
                                y, 
                                z + radius*np.sin(angle*np.pi/180),
                                *orientation)
            )
    #time.sleep(.2)

In [ ]:
for i in range(0, 20):
    response = env.request(SpawnObjectRequest(int(i % 2),
                                ObjectSpawnMethod.RANDOM))

In [ ]:
print(env.request(ObjectsRequest()).metadata)

In [ ]:
# Remove some objects by id
env.request(RemoveObjectsRequest([2,4,6,8,10,12,14,16,18,20]))

In [ ]:
# Remove all objects
env.request(RemoveObjectsRequest())

### Demo Random Seed
Sets the seed, then randomly respawns twice.

Note that this may not be reproducible if a scene or some asset in the scene also uses the Unity Engine's random number generator.

In [ ]:
seed = 34512341

for i in range(2):
    print("Iteration " + str(i) + " results:\n")
    env.send(SetRandomSeed(seed))
    env.request(RemoveObjectsRequest())
    time.sleep(.2)
    env.send(Respawn())
    time.sleep(1)
    env.send(Respawn())
    time.sleep(1)
    response = env.request(SpawnObjectRequest(ObjectType.CUBE,
                                ObjectSpawnMethod.RANDOM))
    print(response.metadata)